<div class="alert alert-info">
Great job guys! Just be sure in the future to think about and discuss the assumptions you are making along the way and how that mihgt impact your interpretation of the results you plot. 55/60
</div>

# Problem 4.2: Caulobacter growth
Attribution: Zhiyang did most of the hw4_2.1, Xinhong did most of the coding of hw4_2.2 and hw4_2.3, the whole group all discuss and give suggestion to each step together.We used and modified some of codes from Justin's notes.
            

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

# A whole bunch of skimage stuff
import skimage
import skimage.feature
import skimage.filters
import skimage.filters.rank
import skimage.io
import skimage.morphology
import skimage.segmentation

# And some useful scipy.ndimage stuff
import scipy.ndimage

import bebi103

import altair as alt

import bokeh
bokeh.io.output_notebook()

### hw4_2.1
To start, we find the all the sequential images from the directory, and see how many images we've got to make sure it is the right directory.

In [ ]:
# The directory containing sequential images of bacteria
data_dir = '../../data/iyer_biswas_et_al/sequential_full_field'

# Glob string for images
im_glob = os.path.join(data_dir, '*.tif')

# Get list of files in directory
im_list = sorted(glob.glob(im_glob))

# Let's look at the entries
im_list[:]

We used the code from last homework, which would load those images as a collection and we also took a look at the length of the list to make sure that all images are loaded.

In [ ]:
# Load in TIFF stack
ic = skimage.io.ImageCollection(im_glob,
                                conserve_memory=False)

# How long is it?
print('There are {0:d} frames.'.format(len(ic)))

Take a look at one of the image to see what it is like:

In [ ]:
ic[0]

Firstly, we want to look at all the images.

In [ ]:
# Define interpixel distance given by the dataset.
ip = 0.052

# Use the frame ID as the name for plots
im_names = '0123456789'

plots = [bebi103.viz.imshow(im,
                            plot_height=300,
                            title=name,
                            interpixel_distance=ip,
                            length_units='µm')
             for name, im in zip(im_names, ic)]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

They don't look that different, so we take one of them to try different image segmentation methods and then apply the best one on all of them to test. We just arbitrarily choose the fifth image to go with. Take a close look, and for the convenience of finding optimal zoom areas (potential ROIs), we plot it with pixel numbers instead of actual scales.

In [ ]:
# Take a copy of the fifth image in the collection.
im = np.copy(ic[4])

plots = [[bebi103.viz.imshow(im,
                            plot_height=500,
                            title='Test copy',
                            interpixel_distance=ip,
                            length_units='pixels')]]

bokeh.io.show(bokeh.layouts.gridplot(plots))

By just looking at this image, we can see some parts that would probably not be easy to segment. For instance, the bottom part has a lot of string-like artifacts, and so does the upper bottom part. There are also a couple of regions where some bacteria cluster together, thus we can give some conditions to choose bacteria for long time course imaging: 

1. Well separated with other bacteria;
2. Away from any types of artifacts;
3. Uniform lighting through single bactrium;
4. The fixed bacteria do not move among frames.

But we will not zoom to some speicific regions now; we plan to do some basic segmentation first to make sure our conditions work as expected.

As we proceed, we need to demonstrate the differences before and after segmentation, so we borrow this nice function from Justin's notes and make some modifications for convenient changes between pixel number and actual scale.

In [ ]:
def show_two_ims(im_1, im_2, titles=[None, None], interpixel_distances=[],
                 color_mapper=None):
    """Convenient function for showing two images side by side."""
    if len(interpixel_distances) == 2:
        p_1 = bebi103.viz.imshow(im_1,
                                 plot_height=300,
                                 title=titles[0],
                                 color_mapper=color_mapper,
                                 interpixel_distance=interpixel_distances[0],
                                 length_units='µm')
        p_2 = bebi103.viz.imshow(im_2,
                                 plot_height=300,
                                 title=titles[1],
                                 color_mapper=color_mapper,
                                 interpixel_distance=interpixel_distances[0],
                                 length_units='µm')
        p_2.x_range = p_1.x_range
        p_2.y_range = p_1.y_range
    else:
        p_1 = bebi103.viz.imshow(im_1,
                                 plot_height=300,
                                 title=titles[0],
                                 color_mapper=color_mapper,
                                 length_units='pixels')
        p_2 = bebi103.viz.imshow(im_2,
                                 plot_height=300,
                                 title=titles[1],
                                 color_mapper=color_mapper,
                                 length_units='pixels')
        p_2.x_range = p_1.x_range
        p_2.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

We did the Gaussian blur first just to check whether the background lighting is uniform or not.

In [ ]:
# Convert image to float
im_float = skimage.img_as_float(im)

# Filter the image with a strong Gaussian blur
im_bg = skimage.filters.gaussian(im_float, 200.0)

bokeh.io.show(show_two_ims(im_float, im_bg, titles=['original', 'background']))

The background is not very uniform, but the purpose of ours is just to segment out the bacteria and we would like to do some threshold anyway, thus we choose not to do the background substration for now. Also, once we find that once converted to float number, the image cannot go through some function that requires uint8 or uint16 data type, like skimage.filter.rank.mean. 

Now, we want to start with the easiest thresholding methods like manually defining the threshould by looking at the histogram of the image, so we plot the histogram first.



In [ ]:
# Compute histograms
hist_bins = skimage.exposure.histogram(im)

def plot_hist(hist_bin, title, y_axis_type='linear'):
    """Make plot of image histogram."""
    p = bokeh.plotting.figure(plot_height=300,
                              plot_width=400,
                              y_axis_type=y_axis_type,
                              x_axis_label='intensity',
                              y_axis_label='count',
                              title=title)
    hist, bins = hist_bin
    p.line(bins, hist, line_width=2)

    return p

# Display histograms
plots = [[plot_hist(hist_bins, 'Histogram','log')]]
bokeh.io.show(bokeh.layouts.gridplot(plots))

It looks like the turnover point is around 4000 where we suppose most of the background would add in, so we choose 4000 as the threshold for the manual thresholding.

In [ ]:
# Threshold image
thesh = 4000
im_bw = im < thesh

# Take a look
bokeh.io.show(show_two_ims(im,
                           im_bw,
                           titles=['original', 'thresholded']))

It looks okay. There are some 'bright' bacteria lost in the processing, but we do not need all of them to show up properly. There are also some artifacts around the edge and some small dots which we can use the clear border and remove small object functions to clean up.

In [ ]:
# Clear the border
im_bw = skimage.segmentation.clear_border(im_bw)

# Remove all the small objects
im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=400)

# Take a look
bokeh.io.show(show_two_ims(im,
                           im_bw,
                           titles=['original', 'thresholded']))

It is actually not bad, though we lose a significant number of bacteria. As expected, those regions that have clustered bacteria or some artifacts cannot show up well in the processed image. However, it is kind of tricky to pick the threshold by just looking at the histogram, so we try to use the function Justin gives in the notes, which would compare each pixel values with its local mean.

In [ ]:
def bebi103_thresh(im, selem, white_true=True, k_range=(0.5, 1.5), min_size=100):
    """
    Threshold image as described above.  Morphological mean filter is 
    applied using selem.
    """    
    # Determine comparison operator
    if white_true:
        compare = np.greater
        sign = -1
    else:
        compare = np.less
        sign = 1
    
    # Do the mean filter
    im_mean = skimage.filters.rank.mean(im, selem)

    # Compute number of pixels in binary image as a function of k
    k = np.linspace(k_range[0], k_range[1], 100)
    n_pix = np.empty_like(k)
    for i in range(len(k)):
        n_pix[i] = compare(im, k[i] * im_mean).sum() 

    # Compute rough second derivative
    dn_pix_dk2 = np.diff(np.diff(n_pix))

    # Find index of maximal second derivative
    max_ind = np.argmax(sign * dn_pix_dk2)

    # Use this index to set k
    k_opt = k[max_ind - sign * 2]

    # Threshold with this k
    im_bw = compare(im, k_opt * im_mean)

    # Remove all the small objects
    im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=min_size)

    return im_bw, k_opt

# Make the structuring element 50 pixel radius disk
selem = skimage.morphology.disk(50)

# Threshhold based on mean filter
im_bw_1, k = bebi103_thresh(im, selem, white_true=False, k_range=(0.5, 0.8), min_size=400)

# Clear the border
im_bw_1 = skimage.segmentation.clear_border(im_bw_1)

bokeh.io.show(show_two_ims(im_float,
                           im_bw_1,
                           titles=['original', 'Auto_thres']))

More bacteria are gone but so are the artifacts, and those meet all of our conditions stay intact. We do play with two parameters a little bit: the size of structuring element and the range of the threshold ratio $k$. The biggest problem we encounter is the tails of bacteria showing up sometimes, so we try to decrease the size of structuring element, which does help but would also lead to some missing pixels inside bacteria. So we go with the 50 pixel radius disk as our structuring element, and tune down the ratio so that the pixels only with high enough contrast, i.e. here those whose intensity is significantly lower than the local mean, to eliminate the tail in segmented images. Due to the tail problem, we also try a similar way to automatically find a good threshold, but instead of comparing the pixel intensity with its local mean, we may compare it with some fixed values.

In [ ]:
def bebi103_thresh_new(im, white_true=True, k_range=[], 
                   min_size=100):
    """
    Absolute threshold finder
    """    
    # Determine comparison operator
    if white_true:
        compare = np.greater
        sign = -1
    else:
        compare = np.less
        sign = 1
    
    if len(k_range) == 2:
        k = np.linspace(k_range[0], k_range[1], 1000)
    else:
        k = np.linspace(im.min(),im.max(),1000)
        
    n_pix = np.empty_like(k)
    for i in range(len(k)):
        n_pix[i] = compare(im, k[i]).sum()
        
    # Compute rough second derivative
    dn_pix_dk2 = np.diff(np.diff(n_pix))
        
    # Find index of maximal second derivative
    max_ind = np.argmax(sign * dn_pix_dk2)
        
    # Use this index to set k
    k_opt = k[max_ind - sign * 2]
        
    # Threshold with this k
    im_bw = compare(im, k_opt)
    
    # Clear the border
    im_bw = skimage.segmentation.clear_border(im_bw)

    # Remove all the small objects
    im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=min_size)

    return im_bw, k_opt


# Threshhold based on mean filter
im_bw_2, k = bebi103_thresh_new(im, white_true=False, k_range=(3600,4000),min_size=400)

bokeh.io.show(show_two_ims(im_float,
                           im_bw_2,
                           titles=['original', 'New_auto_thres']))

This function still requires us to speicify a certain range for the optimal thresholding, and works quite similarily with the manual thresholding. But anyway, we think it is good enough for us to select bacteria for further analysis. We did try other advanced edge finder functions, but they all did not work well probably due to the tail problem. We suspect the reason is that no matter how we tune the parameters, those methods are mostly based on gradient, while the beginning parts of the tails usually have a similar gradient with bacterial bodies. Hence, we decide to go with the basic threshold methods and choose best bacteria based on those images.

Now, we think it is better to choose bacteria that are undergoing division, because this is what we really care about. Choosing those are not dividing would give risk that something weird might happen when they actually divide. Combined with aforementioned conditions, we choose several 'ROIs' to select. For convenience, we write a function to generate our zoom slice from start points and sizs, since the image is actually displayed upside down due to the coordinate problem. 

In [ ]:
def zoom_finder(im, x_start, y_start, size=120):
    """
    Generate zoom slice from start point and size
    """
    _, y = im.shape
    return np.s_[(y - size - y_start):(y - y_start), x_start:(x_start + size)]

And then we locate 3 regions that we think contain the most promising bacteria, the first two of which contains one bacterium of interest, while the last one contains multiple. We would like to further select from the last one after some test segmentation.

In [ ]:
# Zoom start points and sizes
zoom_start = [[590, 260, 120], [715, 375, 120],[125, 250, 400]]

#Generate zoom slices
zooms = []
for i in range(len(zoom_start)):
    zooms.append(zoom_finder(im, zoom_start[i][0], zoom_start[i][1], zoom_start[i][2]))

#Plot out 3 ROIs in actual scale
names = '123'
plots = [bebi103.viz.imshow(im[zoom],
                            plot_height=300,
                            title=name,
                            interpixel_distance=ip,
                            length_units='µm')
             for zoom, name in zip(zooms, names)]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

Let's try to see the segmented versions of those, we write a modified version of show_two_ims.

In [ ]:
def show_4_ims(im_1, im_2, im_3, im_4, titles=[None, None, None, None], interpixel_distances=[ip, ip, ip, ip],
                 color_mapper=None):
    """Convenient function for showing four images side by side."""
    p_1 = bebi103.viz.imshow(im_1,
                             plot_height=250,
                             title=titles[0],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
    p_2 = bebi103.viz.imshow(im_2,
                             plot_height=250,
                             title=titles[1],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[1],
                             length_units='µm')
    p_3 = bebi103.viz.imshow(im_3,
                             plot_height=250,
                             title=titles[2],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[2],
                             length_units='µm')
    p_4 = bebi103.viz.imshow(im_4,
                             plot_height=250,
                             title=titles[3],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[3],
                             length_units='µm')
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    p_3.x_range = p_1.x_range
    p_3.y_range = p_1.y_range
    
    p_4.x_range = p_1.x_range
    p_4.y_range = p_1.y_range
    
    return [p_1, p_2, p_3, p_4]

In [ ]:
title = ["original", "Manual_thresh", "Auto_thresh", "New_auto_thresh"]

p = [show_4_ims(im_float[zoom],
                           im_bw[zoom],
                           im_bw_1[zoom],
                           im_bw_2[zoom],
                           titles=[t + "_" + name for t in title])
     for zoom, name in zip(zooms, names)]

bokeh.io.show(bokeh.layouts.gridplot(p))

They are quite similar, and the thresholding function in the notes actually works slightly better for all of those ROIs, we may see what if we do some opening to separate those dividing cells.

In [ ]:
selem = skimage.morphology.disk(5)

im_open = skimage.morphology.opening(im_bw, selem)
im_open_1 = skimage.morphology.opening(im_bw_1, selem)
im_open_2 = skimage.morphology.opening(im_bw_2, selem)

title = ["original", "Manual_thresh_open", "Auto_thresh_open", "New_auto_thresh_open"]

p = [show_4_ims(im_float[zoom],
                im_open[zoom],
                im_open_1[zoom],
                im_open_2[zoom],
                titles=[t + "_" + name for t in title])
     for zoom, name in zip(zooms, names)]

bokeh.io.show(bokeh.layouts.gridplot(p))

In this case, the auto_thres case separates the bacteria too much which might cause a false positive division. We may use the modified thresholding function to furthor explore among the rest of frames. Now, we convert all the frames into binary images via our modified thresholding function.

In [ ]:
def collection_thres(ic, 
                     white_true=True, 
                     k_range=[], 
                     min_size=100):
    #Initialize a empty list
    ic_seg = []

    # Segment each frame
    for img in ic:
        temp,_ = bebi103_thresh_new(img, white_true=white_true, k_range=k_range,min_size=min_size)
        ic_seg.append(temp)
    
    return ic_seg

ic_seg = collection_thres(ic,
                         white_true=False,
                         k_range=(3600, 4000),
                         min_size=400)

# Take a look    
ic_seg[0]

It looks like a correctly-segmented image, so we proceed and have a function to convert binary images to single-channel color images.

In [ ]:
def bin2gray(im):
    x, y = im.shape
    temp = np.zeros([x, y, 1])
    for (i, j), val in np.ndenumerate(im):
        if val:
            temp[i][j] = 1
    return temp

# Test a little bit
bokeh.io.show(show_two_ims(ic_seg[4],
                           bin2gray(ic_seg[4]),
                           titles=['Binary', 'Single-channel']))

They are the same, so it should be good to integrate into our convertion function for the image collection, and reconvert the image collection.

In [ ]:
def collection_thres_gray(ic,selem,
                     white_true=True, 
                     k_range=[], 
                     min_size=100):
    #Initialize a empty list
    ic_seg = []

    # Segment each frame
    for img in ic:
        temp,_ = bebi103_thresh_new(img, white_true=white_true, k_range=k_range,min_size=min_size)
        # Open to separate dividing bacteria
        temp = skimage.morphology.opening(temp, selem)
        ic_seg.append(bin2gray(temp))
    
    return ic_seg

selem = skimage.morphology.disk(5)

ic_seg = collection_thres_gray(ic,selem,
                         white_true=False,
                         k_range=(3600, 4000),
                         min_size=400)

# Take a look    
ic_seg[0].shape

It should be good, so we can proceed and add all the converted image together, supposing the scale and layout don't change over frames, which should be true. In this case, we can have sort of a distribution of how many frames share pixels after thresholding. We have some trouble tuning the color mapper and colorbar, but as the defalut color mapper goes, yellow indicates high pixel density and blue indicates low. We can then take a look at those zoom slices we choose.

In [ ]:
ic_sum = np.array(ic_seg).sum(axis=0)


#Plot out 3 ROIs in actual scale
names = '123'
plots = [bebi103.viz.imshow(ic_sum[zoom],
                            plot_height=300,
                            title=name,
                            interpixel_distance=ip,
                            length_units='µm')
             for zoom, name in zip(zooms, names)]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))

The first two looks fine, and from the third one, we can still pick out a couple of bacteria that fulfills our conditions, while in the third one, those are mostly close to one or multiple bacteria, which may or may not lead to error when we do long time course analysis, because they could probably be eliminated by making them close to the edge and then clearing border. However, being too close to other bacteria might also lead to problems like failure to flowing away the divden cells. Thus, we choose the best one from the thrid zoom slice and remake the slices for final selection.

In [ ]:
zoom_start_final = [[590, 260, 120], [715, 375, 120],[200, 500, 100]]

#Generate zoom slices
zooms = []
for i in range(len(zoom_start)):
    zooms.append(zoom_finder(im, zoom_start_final[i][0], zoom_start_final[i][1], zoom_start_final[i][2]))
                    
plots = [bebi103.viz.imshow(ic_sum[zoom],
                            plot_height=300,
                            title=name,
                            interpixel_distance=ip,
                            length_units='µm')
             for zoom, name in zip(zooms, names)]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=3))                    

In all, our criteria for choosing bacteria for long time course imaging could be concluded as:

1. Well separated with other bacteria;
2. Away from any types of artifacts;
3. Uniform lighting through single bactrium;
4. The fixed bacteria do not move among frames.

The first one is to avoid mislabelling and other problems that could happen when multiple cells are imaged; the second and third one is to make sure our segmentation is significantly affected through the imaging process. We use a modified thresholding function and then select bacteria that fulfill those first three conditions, and use the overlayed segmented image of all the frames to find bacteria that do not move through all frames, since we do not want part of the imaged cell move out of our ROI sometime if it is not well anchored.

## hw4_2.2 and hw4_2.3

Let's load in 'bacterium_1.tif' and check the information of it

In [ ]:
# Load in TIFF stack
fname = '../../data/iyer_biswas_et_al/bacterium_1.tif'
ic = skimage.io.ImageCollection(fname, conserve_memory=False)[0]

# How long is it?
print('There are {0:d} frames.'.format(len(ic)))

# Let's get information about the image
print(type(ic), ic.dtype, ic.shape)

We already know 'The frame rate is 1 frame per minute. The interpixel spacing is 0.052 µm'

In [ ]:
# add time stamps to our image (frame rate: 1 frame/min)
fps = 1
t = np.arange(0, len(ic)) / fps

# interpixel is 0.052 µm
ip = 0.052

It would be nice to have a function to plot two images side by side for comparision, this is the function from Justin's Tutorial

In [ ]:
# define a function from tutorial to display 2 images 
def show_two_ims(im_1, im_2, titles=[None, None], interpixel_distances=[0.13, 0.13],
                 color_mapper=None):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.viz.imshow(im_1,
                             plot_height=300,
                             title=titles[0],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
    p_2 = bebi103.viz.imshow(im_2,
                             plot_height=300,
                             title=titles[1],
                             color_mapper=color_mapper,
                             interpixel_distance=interpixel_distances[0],
                             length_units='µm')
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range
    
    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

In [ ]:
# Take a look at the first and second time point to get an idea of what it looks like
im = ic[0,:,:]
im1 = ic[1,:,:]

# Take a look
bokeh.io.show(show_two_ims(im,
                           im1,
                           titles=['im', 'im1']))

Let's start the thresholding process and extract the bacteria area.
We decide to go for the thresh function in the tutorial that help you find the right threshold. (We also try sober filtering and other method to do the edge detection and segmentation, but find that the find threshold method works the best)  
But we still need to play around with the k_range to get a better segmentation. 
Let's define the find threshold function first.

In [ ]:
# Use this find threshold function from tutorial
def bebi103_thresh(im, selem, white_true=True, k_range=(0.5, 2), 
                   min_size=120):
    """
    Threshold image as described above.  Morphological mean filter is 
    applied using selem.
    """    
    # Determine comparison operator
    if white_true:
        compare = np.greater
        sign = -1
    else:
        compare = np.less
        sign = 1
    
    # Do the mean filter
    im_mean = skimage.filters.rank.mean(im, selem)

    # Compute number of pixels in binary image as a function of k
    k = np.linspace(k_range[0], k_range[1], 100)
    n_pix = np.empty_like(k)
    for i in range(len(k)):
        n_pix[i] = compare(im, k[i] * im_mean).sum() 

    # Compute rough second derivative
    dn_pix_dk2 = np.diff(np.diff(n_pix))

    # Find index of maximal second derivative
    max_ind = np.argmax(sign * dn_pix_dk2)

    # Use this index to set k
    k_opt = k[max_ind - sign * 2]

    # Threshold with this k
    im_bw = compare(im, k_opt * im_mean)

    # Remove all the small objects
    im_bw = skimage.morphology.remove_small_objects(im_bw, min_size=min_size)
    
    #Clear the border
    im_bw = skimage.segmentation.clear_border(im_bw)

    return im_bw, k_opt

After defining the function, we need to play around with the k_range to find the suitable segmentation. We use the image of first time point to test different k_range.

In [ ]:
# after test different k_range, we find that k_range=(0.5, 2) gives us the best segmentation
im_thresh, k = bebi103_thresh(im, 
                              skimage.morphology.disk(100), 
                              white_true=False, 
                              k_range=(0.5, 2),
                              min_size=100)

# Show result
bokeh.io.show(
    bebi103.viz.imshow(im_thresh, 
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('gray'),
                       interpixel_distance=ip,
                       length_units='µm'))



This segmentation seems nice. So let's go ahead and try to get the properties of the bacterium area from this image.  
To get properties, we need to label the binary image first

In [ ]:
# Label binary image; backward kwarg says value in im_bw to consider backgr.
im_labeled, n_labels = skimage.measure.label(
                            im_thresh, background=0, return_num=True)

# See result (one of the few times it's ok to use rainbow colormap!)
bokeh.io.show(
    bebi103.viz.imshow(im_labeled, 
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('rainbow'),
                       interpixel_distance=ip,
                       length_units='µm'))

In [ ]:
# Try getting properties of this time point 
im_y_props = skimage.measure.regionprops(im_labeled, intensity_image=im)

# put the data into a tidy data frame
data = [[prop.label, prop.area, prop.mean_intensity] for prop in im_y_props]
df = pd.DataFrame(data=data, columns=['label', 'area (sq ip distance)', 'mean intensity (a.u.)'])
df['area (sq µm)'] = df['area (sq ip distance)'] * ip**2

# Take a look
df

We get the properties from the bacterium area succesfully, so let's go ahead and write a function that could process all the image(time points) and get back the properties of each time point.

In [ ]:
# define a function for processing every image and return area/intensity
def process_im(im):
    # Thresholding the image
    im_thresh, k = bebi103_thresh(im, 
                                  skimage.morphology.disk(100), 
                                  white_true=False, 
                                  min_size=100)
    
    # Label binary image
    im_labeled, n_labels = skimage.measure.label(
                                    im_thresh, background=0, return_num=True)
    
    # Get properties of this time point 
    im_y_props = skimage.measure.regionprops(im_labeled, intensity_image=im)
    
    # put the data into a tidy data frame
    data = [[prop.label, prop.area, prop.mean_intensity] for prop in im_y_props]
    df = pd.DataFrame(data=data, columns=['label', 'area (sq ip distance)', 'mean intensity (a.u.)'])
    df['area (sq µm)'] = df['area (sq ip distance)'] * ip**2

    # return df
    return df


In [ ]:
# Check whether our function works using the first time point
df0 = process_im(ic[0,:,:])
df0

It looks good when we test our function on the first time point, so let's run this function on each time point of our experiment.  
To store all the properties, we need to create an empty dataframe to store all the return data from the function.

In [ ]:
# Create an empty dataframe to store the properties
df = pd.DataFrame(columns=['label', 'area (sq ip distance)', 'mean intensity (a.u.)', 'area (sq µm)'])

# Look through and add properties of each time point to the empty dataframe
for i, im in enumerate(ic):
    df1 = process_im(ic[i,:,:])
    df = df.append(df1, ignore_index = True)
    
df.head()

From the dataframe, we could see the 'label' in some row is 2, which means the function detect 2 cells after segmentation, let's check what is going on with those frames.

In [ ]:
# Check whether there are some frames are labeled as more than 1 cell
df.loc[(df['label']!=1)]

Let's check some of those frames to see what is going on.

In [ ]:
# check one of them see what's going on
im_thresh1, k = bebi103_thresh(ic[100,:,:], 
                               skimage.morphology.disk(100), 
                               white_true=False,
                               min_size=100)

# Show result
bokeh.io.show(
    bebi103.viz.imshow(im_thresh1, 
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('gray'),
                       interpixel_distance=ip,
                       length_units='µm'))

We could see the generated cell are still hanging around which leads to the problem that there 2 labled cells are detected. But we could see the division is already complete, so it's better for our analysis if we only consider one of the cell in this situation.  
In order to do that, let's write a new process function that drop the extra labeled area and keep only one labeled area.


In [ ]:
# define a new function for processing every image and return area/intensity
# new process function that drop the extra labeled area and keep only one labeled area.
def new_process_im(im, i, k_range):
    # Thresholding the image
    im_thresh, k = bebi103_thresh(im, 
                                  skimage.morphology.disk(100), 
                                  white_true=False, 
                                  k_range=k_range,
                                  min_size=100)
    
    # Label binary image
    im_labeled, n_labels = skimage.measure.label(
                            im_thresh, background=0, return_num=True)
    
    # Get properties of this time point 
    im_y_props = skimage.measure.regionprops(im_labeled, intensity_image=im)
    
    # put the data into a tidy data frame
    data = [[prop.label, prop.area, prop.mean_intensity] for prop in im_y_props]
    df = pd.DataFrame(data=data, 
                      columns=['label', 'area (sq ip distance)', 'mean intensity (a.u.)'])
    
    # drop second labeled cell when there are 2 labled cells
    if len(df) > 1:
        df = df.drop([1])
        
    # add the actual area of bacteria
    df['area (sq µm)'] = df['area (sq ip distance)'] * ip**2
    
    # add the time to the dataframe
    df['t'] = i
    
    # return df
    return df

Run this new process function on all the time points.

In [ ]:
# Create an empty dataframe to store the properties
df_new = pd.DataFrame(columns=['label', 
                               'area (sq ip distance)', 
                               'mean intensity (a.u.)', 
                               'area (sq µm)'])

# Look through and add properties of each time point to the empty dataframe
for i, im in enumerate(ic):
    df1 = new_process_im(ic[i,:,:], i, (0.5, 2))
    df_new = df_new.append(df1, ignore_index=True)
    
df_new

When we look at this dataset, we notice that the time goes from 0-1899s (consistent with the information that it has 1900 frames), but you only get 1890 rows, which mean the threshold function didn't detect anything in 9 of the frames, we should keep that in mind in the following analysis. Maybe we need extra operation to deal with these missing time points.

But let's do a scatter plot on these area data first. 

In [ ]:
# Function to make scatter plot
def scatter(df, x, y, p=None, color='#1f77b4', plot_height=450, plot_width=1500):
    """Populate a figure with a scatter plot."""
    if p is None:
        p = bokeh.plotting.figure(plot_height=plot_height, 
                                  plot_width=plot_width, 
                                  x_axis_label=x,
                                  y_axis_label=y)
        
    p.circle(df[x], df[y], color=color)
    
    return p

In [ ]:
# Plot the change of the area
bokeh.io.show(scatter(df_new, 't', 'area (sq µm)'))

Although there are some noises between, but the general dynamics of the bacterium area change looks reasonable, so let's go ahead and define time of division.  

When a division happens, since the generated cell would be washed away, we would expect to see the dramatic drop of area. Idealy, the drop of the area would be half of the original area (let's assume original bacteria and generated bacteria are exactly same size, so we would expect a 50% drop), but in reality, the original bacteria and generated one has some small difference, so let's expect a 30% drop (this threshold is defined by ourselves, you would need to have a tradeoff between missing division events when setting a high standard for area drop and using a low stantard thus overcount division events)

Besides dramatic drop, the other property that we expect to see at division is the cell would grow again after this division event.

In general, we would like to define the time of division as the time point when there is a >30% area drop and the cell grow again in next consecutive time point. In order to implement this definition, we need to calculate the area difference between frames. 

When we calculate the the area difference between frames using the dataframe we generated before, we need to keep in mind the issue we mentioned before, there are some null detection in some frames. So when we did the calculation for area difference, we need to df_new['area_difference']/df_new['time_difference'] to get the real df_new['area_difference'] between consecutive frames.

In [ ]:
# Calculate the area_difference and time_difference
df_new['area_difference'] = df_new['area (sq µm)'] - df_new['area (sq µm)'].shift(-1)
df_new['time_difference'] = df_new['t'].shift(-1) - df_new['t']

# Get the area_difference between conseceutive frame
df_new['area_difference'] = df_new['area_difference']/df_new['time_difference']

# Caculate the percentage of area change
df_new['area_change_diff'] = df_new['area_difference']/df_new['area (sq µm)'] 

# Define the time of division as the time point when there is a >30% area drop 
# and the cell grows again in next consecutive time point
df_new['division'] = ((df_new['area_change_diff'] > 0.3) & 
                      (df_new['area_change_diff'].shift(-1) < 0))

df_new.head()

In [ ]:
# store the data to csv for further use
df_new.to_csv('bacterium_1.csv', sep='\t')

We would like to slice out those that are defined as 'division time point' for further analysis

In [ ]:
# Slice out the time point defined as 'division'
df_new_div = df_new.loc[df_new['division'] == True, :]
df_new_div = df_new_div.reset_index()
df_new_div

In [ ]:
# Take a look at certain time point to verify the above result
# Look at t=98 and t=99 for example because 98s is indicated as division
im98 = ic[98,:,:]
im99 = ic[99,:,:]

# Take a look
bokeh.io.show(show_two_ims(im98,
                           im99,
                           titles=['im98', 'im99']))

We could see that they indeed divide at t=98s!

In [ ]:
# Look at t=1621 and t=1622 for example because 1621s is indicated as division
im1621 = ic[1621,:,:]
im1622 = ic[1622,:,:]

# Take a look
bokeh.io.show(show_two_ims(im1621,
                           im1622,
                           titles=['im1621', 'im1622']))

We could see that they indeed divide at t=1621s!

We are interested in the time_interval between these division time points. Let's add an ID for each division event, and caculate the time interval between each consecutive event.

In [ ]:
# Add ID to each division event
df_new_div['division_ID'] = range(0, 0+len(df_new_div))

# Calculate the time interval between division event
df_new_div['division_interval (min)'] = df_new_div['t'].shift(-1) - df_new_div['t'] 

df_new_div

In [ ]:
# scatter plot the time interval between 2 consecutive time point
bokeh.io.show(scatter(df_new_div, 'division_ID', 'division_interval (min)', 
                      plot_height=250, plot_width=800))

From the scatter plot, we could see that there is not a clear trend of division interval change along the time, it's quite persistent regarding the time.

In [ ]:
# plot ECDF of the time interval between 2 consecutive time point
bokeh.io.show(bebi103.viz.ecdf(df_new_div['division_interval (min)'],
                               x_axis_label='division_interval (min)'))

From the ECDF, we could see most of the division interval lies at around 100min, indicating each division of this bacteria takes around 100min.

### Do the similar manipulation on the bacterium_2.tif

In [ ]:
# Load in TIFF stack
fname1 = '../../data/iyer_biswas_et_al/bacterium_2.tif'
ic1 = skimage.io.ImageCollection(fname1, conserve_memory=False)[0]

# How long is it?
print('There are {0:d} frames.'.format(len(ic1)))

# Let's get information about the image
print(type(ic1), ic1.dtype, ic1.shape)


In [ ]:
# Take a look at the first and second time point to get an idea of what it looks like
im = ic1[0,:,:]
im1 = ic1[1,:,:]

# Take a look
bokeh.io.show(show_two_ims(im,
                           im1,
                           titles=['im', 'im1']))

In [ ]:
# Play around the k_range in the threshold function to decide the suitable k_range
# Find that k_range=(0.5, 2.6) works fine

im_thresh111, k = bebi103_thresh(ic1[0,:,:], 
                                 skimage.morphology.disk(100), 
                                 white_true=False, 
                                 k_range=(0.5, 2.6),
                                 min_size=100)

# Show result
bokeh.io.show(
    bebi103.viz.imshow(im_thresh111, 
                       color_mapper=bebi103.viz.mpl_cmap_to_color_mapper('gray'),
                       interpixel_distance=ip,
                       length_units='µm'))

In [ ]:
# Create an empty dataframe to store the properties
df_new1 = pd.DataFrame(columns=['label', 
                                'area (sq ip distance)', 
                                'mean intensity (a.u.)', 
                                'area (sq µm)'])

# Look through and add properties of each time point to the empty dataframe
for i, im in enumerate(ic1):
    df1 = new_process_im(ic1[i,:,:], i, k_range=(0.5, 2.6))
    df_new1 = df_new1.append(df1, ignore_index = True)
    
df_new1

In [ ]:
bokeh.io.show(scatter(df_new1, 't', 'area (sq µm)'))

In [ ]:
# define time of division
df_new1['area_difference'] = df_new1['area (sq µm)'] - df_new1['area (sq µm)'].shift(-1)
df_new1['time_difference'] = df_new1['t'].shift(-1) - df_new1['t']
df_new1['area_difference'] = df_new1['area_difference']/df_new1['time_difference']
df_new1['area_change_diff'] = df_new1['area_difference']/df_new1['area (sq µm)'] 

# define division when the percentage of area drop > 30%
df_new1['division'] = ((df_new1['area_change_diff'] > 0.3) & 
                       (df_new1['area_change_diff'].shift(-1) < 0))

In [ ]:
# store the data to csv for further use
df_new1.to_csv('bacterium_2.csv', sep='\t')

In [ ]:
# Slice out the time point defined as 'division'
df_new_div1 = df_new1.loc[df_new1['division'] == True, :]
df_new_div1 = df_new_div1.reset_index()

# Add division ID to dataframe
df_new_div1['division_ID'] = range(0, 0+len(df_new_div1))

# Calculate the time interval between division
df_new_div1['division_interval (min)'] = df_new_div1['t'].shift(-1) - df_new_div1['t'] 

df_new_div1.head()

In [ ]:
# scatter plot the time interval between 2 consecutive time point
bokeh.io.show(scatter(df_new_div1, 'division_ID', 'division_interval (min)', plot_height=250, plot_width=800))

From this scatter plot of the division interval of the bacteria2, we could see there are lots of them lies around zero, that is pretty wierd, let's look closely at some of the frames defined as 'division time point' to figure out what's going on. We notice 188s and 192s are both defined as 'division time point' and they are very close in time, so let's look at those frames.

In [ ]:
# Take a look at the 188s-192s to get an idea of what's going on
im188 = ic1[188,:,:]
im189 = ic1[189,:,:]

# Take a look
bokeh.io.show(show_two_ims(im188,
                           im189,
                           titles=['im188', 'im189']))

In [ ]:
# Take a look at the 188s-192s to get an idea of what's going on
im191 = ic1[191,:,:]
im192 = ic1[192,:,:]

# Take a look
bokeh.io.show(show_two_ims(im191,
                           im192,
                           titles=['im191', 'im192']))

From these images, we could see the generated cell hangs around, and the situation here is also more complicated than what we encounter when dealing with bacteria 1, because this generated is touching the original cell when it flows around, thus the labeling function detect them as one (the flow around one is detected but always detected as a seperate cell when we delt wih bacteria 1, so we could just get rid of the extra detected one at that time), considering this situation, and considering the fact that most of the time when we get time interval <10s is because of this situation, we decided to only keep the one that has time interval >10.

In [ ]:
# Slice out the time point defined as 'division'
df_new_div11 = df_new1.loc[df_new1['division'] == True, :]
df_new_div11 = df_new_div11.reset_index()

# Calculate the time interval between division
df_new_div11['division_interval (min)'] = df_new_div11['t'].shift(-1) - df_new_div11['t'] 
df_new_div11 = df_new_div11.loc[df_new_div11['division_interval (min)'] > 10, :]

# Add division ID to dataframe
df_new_div11['division_ID'] = range(0, 0+len(df_new_div11))

df_new_div11.head()

In [ ]:
# scatter plot the time interval between 2 consecutive time point
bokeh.io.show(scatter(df_new_div11, 'division_ID', 'division_interval (min)', 
                      plot_height=250, plot_width=800))

In [ ]:
bokeh.io.show(bebi103.viz.ecdf(df_new_div11['division_interval (min)'],
                               x_axis_label='division_interval (min)'))

From the ECDF Plot and the scatter plot, we could see that the most of the division interval of bacteria 2 lies at around 100min, which is similar to what we see in bacteria 1. In general, the distribution of division interval looks like a gaussian distribution. And there is no time-related trend of the change in division interval. And we are super curious to see how we could use these datas to do parameter estimation in the following classes.

In [ ]:
%load_ext watermark

In [ ]:
%watermark -v -p numpy,pandas,scipy,skimage,bokeh,bebi103,jupyterlab